Directional Trading Strategy

Downtrend (Underlying's price < 50 day SMA)
 1. If %K > 60 (overbought) and %K drops 5% then:
 2. Buy a >= 45 DTE 1 or 2 ITM Put with OI >= 100 and priced at bid-ask mid-point*
 3. Exit the position, sell the Put, before expiration when:
    a. Put price drops 50%
    b. Underlying's price > previous day's High & %K < 40 (i.e. is oversold)
    c. Sell half of position if Put price gains 80%
    d. Sell at time T (maximum holding period with no gain or loss)

Uptrend (Underlying's price > 50 day SMA)
1. If %K < 40 (oversold) and %K gains 5% then:
2. Buy a >= 45 DTE 1 or 2 ITM Call with OI >= 100 and priced at bid-ask mid-point*
3. Exit the position, sell the Call, before expiration when:
    a. Call price drops 50%
    b. Underlying's price < previous day's Low & %K > 60 (overbought)
    c. Sell half of position if Call price gains 80%
    d. Sell at time T (maximum holding period with no gain or loss)

    *Risk Management: Option Price * 100 <= 5% of trading capital

In [8]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import numpy as np
import pandas as pd
import os

# Load datasets
equity_price_series = pd.read_csv('ge.stock.20080101.1231.csv', index_col='date')

In [9]:
equity_price_series.head()

,open,high,low,last,volume,option volume,implied vol
date,,,,,,,
2008-01-02,37.10,37.45,36.55,36.76,38799202,60299,0.258324
2008-01-03,36.81,37.22,36.70,36.80,26800382,67025,0.250877
2008-01-04,36.54,36.67,35.98,36.04,40729626,83956,0.276142
2008-01-07,36.21,36.50,35.91,36.18,47509039,53254,0.293888
2008-01-08,36.43,36.43,35.20,35.40,44824259,75982,0.316222


In [39]:
highs = equity_price_series.iloc[:, 1]
lows = equity_price_series.iloc[:, 2]
close = equity_price_series.iloc[:, 3]

H14 = highs.rolling(14).max()
L14 = lows.rolling(14).min()
equity_price_series['K'] = 100*(close-L14)/(H14-L14)

In [51]:
equity_price_series['mid'] = (equity_price_series.iloc[:,1]-equity_price_series.iloc[:,2])/2+equity_price_series.iloc[:,2]

In [53]:
equity_price_series['50sma'] = equity_price_series['mid'].rolling(50).mean()

In [54]:
equity_price_series

,open,high,low,last,volume,option volume,implied vol,K,mid,50sma
date,,,,,,,,,,
2008-01-02,37.10,37.45,36.55,36.76,38799202,60299,0.258324,NaN,37.000,NaN
2008-01-03,36.81,37.22,36.70,36.80,26800382,67025,0.250877,NaN,36.960,NaN
2008-01-04,36.54,36.67,35.98,36.04,40729626,83956,0.276142,NaN,36.325,NaN
2008-01-07,36.21,36.50,35.91,36.18,47509039,53254,0.293888,NaN,36.205,NaN
2008-01-08,36.43,36.43,35.20,35.40,44824259,75982,0.316222,NaN,35.815,NaN
...,...,...,...,...,...,...,...,...,...,...
2008-12-24,16.10,16.23,15.92,16.11,23409169,27415,0.537966,10.140845,16.075,17.5699
2008-12-26,16.05,16.13,15.78,15.97,29654708,46953,0.554954,6.197183,15.955,17.4919
2008-12-29,15.95,16.00,15.35,15.66,55804550,70844,0.606879,9.365559,15.675,17.4227


In [58]:
i = 0
equity_price_series['trend'] = None

while i < len(equity_price_series):
    if equity_price_series.iloc[i, 8] > equity_price_series.iloc[i, 9]:
        equity_price_series.iloc[i, 10] = 'up'
    elif equity_price_series.iloc[i, 8] < equity_price_series.iloc[i, 9]:
        equity_price_series.iloc[i, 10] = 'down'
    i = i + 1

In [60]:
equity_price_series.tail()

,open,high,low,last,volume,option volume,implied vol,K,mid,50sma,trend
date,,,,,,,,,,,
2008-12-24,16.10,16.23,15.92,16.11,23409169,27415,0.537966,10.140845,16.075,17.5699,down
2008-12-26,16.05,16.13,15.78,15.97,29654708,46953,0.554954,6.197183,15.955,17.4919,down
2008-12-29,15.95,16.00,15.35,15.66,55804550,70844,0.606879,9.365559,15.675,17.4227,down
2008-12-30,15.80,15.85,15.55,15.82,57449351,99902,0.590034,15.161290,15.700,17.3402,down
2008-12-31,15.82,16.34,15.77,16.20,60224914,78096,0.524874,30.575540,16.055,17.2633,down


In [64]:
i = 0
equity_price_series['option2buy'] = None

while i < len(equity_price_series):
    if equity_price_series.iloc[i, 10] == 'up': 
        if equity_price_series.iloc[i, 7] <= 40:
            equity_price_series.iloc[i, 11] = 'call'
    elif equity_price_series.iloc[i, 10] == 'down': 
        if equity_price_series.iloc[i, 7] >= 60:
            equity_price_series.iloc[i, 11] = 'put'
    i = i + 1

In [70]:
equity_price_series[48:100]

,open,high,low,last,volume,option volume,implied vol,K,mid,50sma,trend,option2buy
date,,,,,,,,,,,,
2008-03-12,33.45,34.46,33.45,33.96,70642944,120363,0.264458,82.206406,33.955,NaN,None,None
2008-03-13,33.51,34.51,33.31,34.25,64523914,150731,0.263034,90.909091,33.910,34.4604,down,put
2008-03-14,34.52,34.57,33.06,33.82,73097516,143860,0.336645,74.315068,33.815,34.3967,down,put
2008-03-17,32.99,34.73,32.83,34.33,80415892,88161,0.345919,87.012987,33.780,34.3331,down,put
2008-03-18,34.91,36.22,34.71,36.14,83668824,184883,0.328818,98.249453,35.465,34.3159,up,None
2008-03-19,36.20,36.63,35.55,35.59,64817457,106356,0.289941,79.116466,36.090,34.3136,up,None
2008-03-20,36.54,37.74,36.40,37.49,110630359,193933,0.281720,95.894910,37.070,34.3387,up,None
2008-03-24,37.63,37.67,37.04,37.40,46992147,130230,0.276616,94.417077,37.355,34.3749,up,None
2008-03-25,37.25,37.53,36.92,37.27,44998693,89819,0.265928,92.282430,37.225,34.4034,up,None


In [88]:
i = 0
j = 0

equity_price_series['K_diff'] = None

while i < len(equity_price_series):
    if i >= 14:
        equity_price_series.iloc[i, 13] = equity_price_series.iloc[i, 7] - equity_price_series.iloc[j, 7]
    i = i + 1
    j = i - 1

In [90]:
i = 0
equity_price_series['signal'] = None

while i < len(equity_price_series):
    if equity_price_series.iloc[i, 11] == 'put': 
        if equity_price_series.iloc[i, 13] <= -5:
            equity_price_series.iloc[i, 12] = 'buy_put'
    elif equity_price_series.iloc[i, 11] == 'call': 
        if equity_price_series.iloc[i, 13] >= 5:
            equity_price_series.iloc[i, 12] = 'buy_call'
    i = i + 1

In [95]:
equity_price_series[48:100]

,open,high,low,last,volume,option volume,implied vol,K,mid,50sma,trend,option2buy,signal,K_diff
date,,,,,,,,,,,,,,
2008-03-12,33.45,34.46,33.45,33.96,70642944,120363,0.264458,82.206406,33.955,NaN,None,None,None,17.63076
2008-03-13,33.51,34.51,33.31,34.25,64523914,150731,0.263034,90.909091,33.910,34.4604,down,put,None,8.702685
2008-03-14,34.52,34.57,33.06,33.82,73097516,143860,0.336645,74.315068,33.815,34.3967,down,put,buy_put,-16.594022
2008-03-17,32.99,34.73,32.83,34.33,80415892,88161,0.345919,87.012987,33.780,34.3331,down,put,None,12.697919
2008-03-18,34.91,36.22,34.71,36.14,83668824,184883,0.328818,98.249453,35.465,34.3159,up,None,None,11.236466
2008-03-19,36.20,36.63,35.55,35.59,64817457,106356,0.289941,79.116466,36.090,34.3136,up,None,None,-19.132987
2008-03-20,36.54,37.74,36.40,37.49,110630359,193933,0.281720,95.894910,37.070,34.3387,up,None,None,16.778444
2008-03-24,37.63,37.67,37.04,37.40,46992147,130230,0.276616,94.417077,37.355,34.3749,up,None,None,-1.477833
2008-03-25,37.25,37.53,36.92,37.27,44998693,89819,0.265928,92.282430,37.225,34.4034,up,None,None,-2.134647


In [97]:
equity_price_series.to_csv('ge.option_trading_signals.2008.csv')

Conditions to enter and exit positions:

Downtrend (Underlying's price < 50 day SMA)
 1. If %K > 60 (overbought) and %K drops 5% then:
 2. Buy a >= 45 DTE 1 or 2 ITM Put with OI >= 100 and priced at bid-ask mid-point*
 3. Exit the position, sell the Put, before expiration when:
    a. Put price drops 50%
    b. Underlying's price > previous day's High & %K < 40 (i.e. is oversold)
    c. Sell half of position if Put price gains 80%
    d. Sell at time T (maximum holding period with no gain or loss)

Uptrend (Underlying's price > 50 day SMA)
1. If %K < 40 (oversold) and %K gains 5% then:
2. Buy a >= 45 DTE 1 or 2 ITM Call with OI >= 100 and priced at bid-ask mid-point*
3. Exit the position, sell the Call, before expiration when:
    a. Call price drops 50%
    b. Underlying's price < previous day's Low & %K > 60 (overbought)
    c. Sell half of position if Call price gains 80%
    d. Sell at time T (maximum holding period with no gain or loss)

    *Risk Management: Option Price * 100 <= 5% of trading capital

In [1]:
import pandas as pd

options_chain = pd.read_csv('ge.options.20080101.1231.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
equity_price_series = pd.read_csv('ge.option_trading_signals.2008.csv', index_col=0, parse_dates=True, infer_datetime_format=True)

In [2]:
from datetime import datetime, date, time, timedelta

for expiry in options_chain.iloc[:, 2]:
    expiration_dates = datetime.strptime(expiry, '%Y-%m-%d')
    
options_chain['expiration_dates'] = expiration_dates

In [4]:
options_chain.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 28894 entries, 2008-01-02 to 2008-12-31
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0    symbol           28894 non-null  object        
 1    under            28894 non-null  object        
 2    expiration       28894 non-null  object        
 3    strike           28894 non-null  float64       
 4    put/call         28894 non-null  object        
 5    bid              28894 non-null  float64       
 6    ask              28894 non-null  float64       
 7    price            28894 non-null  float64       
 8    volume           28894 non-null  int64         
 9    open interest    28894 non-null  int64         
 10   implied vol      26880 non-null  float64       
 11   delta            28894 non-null  float64       
 12   gamma            28894 non-null  float64       
 13   rho              28894 non-null  float64       
 14   thet

In [5]:
equity_price_series.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 253 entries, 2008-01-02 to 2008-12-31
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0    open           253 non-null    float64
 1    high           253 non-null    float64
 2    low            253 non-null    float64
 3    last           253 non-null    float64
 4    volume         253 non-null    int64  
 5    option volume  253 non-null    int64  
 6    implied vol    253 non-null    float64
 7   K               240 non-null    float64
 8   mid             253 non-null    float64
 9   50sma           204 non-null    float64
 10  trend           204 non-null    object 
 11  option2buy      44 non-null     object 
 12  signal          12 non-null     object 
 13  K_diff          239 non-null    float64
dtypes: float64(9), int64(2), object(3)
memory usage: 29.6+ KB


In [4]:
# todo: select contracts from options chain that match the buy signal

In [32]:
options = options_chain.loc[(options_chain.index=='2008-03-04') & (options_chain[' put/call']=='P')]

In [33]:
options.head()

,symbol,under,expiration,strike,put/call,bid,ask,price,volume,open interest,implied vol,delta,gamma,rho,theta,vega,nonstd,expiration_dates
date,,,,,,,,,,,,,,,,,,
2008-03-04,GEOC,GE,2008-03-22,31.0,P,0.09,0.10,0.10,766,2133,0.284217,-0.095971,0.082823,-0.001543,-0.010298,0.012319,0,2010-01-16
2008-03-04,GEOF,GE,2008-03-22,30.0,P,0.04,0.05,0.05,747,20293,0.313007,-0.046852,0.043240,-0.000752,-0.006521,0.007083,0,2010-01-16
2008-03-04,GEOG,GE,2008-03-22,35.0,P,1.56,1.60,1.56,1460,46231,0.186751,-0.853566,0.169831,-0.014062,-0.009115,0.016598,0,2010-01-16
2008-03-04,GEON,GE,2008-03-22,34.0,P,0.85,0.88,0.86,2592,25467,0.204530,-0.617923,0.257765,-0.010050,-0.016596,0.027590,0,2010-01-16
2008-03-04,GEOO,GE,2008-03-22,36.0,P,2.46,2.50,2.49,184,5502,0.271538,-0.882720,0.100201,-0.014987,-0.011370,0.014239,0,2010-01-16


In [55]:
signals = equity_price_series.loc[(equity_price_series['signal']=='buy_put')|(equity_price_series['signal']=='buy_call')]

In [56]:
signals.head()

,open,high,low,last,volume,option volume,implied vol,K,mid,50sma,trend,option2buy,signal,K_diff
date,,,,,,,,,,,,,,
2008-03-14,34.52,34.57,33.06,33.82,73097516,143860,0.336645,74.315068,33.815,34.3967,down,put,buy_put,-16.594022
2008-04-10,36.40,37.07,36.16,36.75,44175033,124576,0.330186,25.000000,36.615,35.0452,up,call,buy_call,15.043290
2008-05-05,33.36,33.36,32.96,33.18,26460612,54290,0.216213,71.084337,33.160,34.4187,down,put,buy_put,-13.749786
2008-05-06,33.07,33.17,32.62,33.00,41743801,104875,0.237848,60.240964,32.895,34.3984,down,put,buy_put,-10.843373
2008-07-21,28.12,28.18,27.61,27.69,40998771,90848,0.263080,74.113475,27.895,29.2826,down,put,buy_put,-10.992908


In [89]:
# filter options contracts for buy signal
i = 0

options = pd.DataFrame()
dates = signals.index

while i < len(signals):
    options = options.append(options_chain.loc[(options_chain.index==dates[i])])
    i = i + 1

In [92]:
options.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1382 entries, 2008-03-14 to 2008-12-11
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0    symbol           1382 non-null   object        
 1    under            1382 non-null   object        
 2    expiration       1382 non-null   object        
 3    strike           1382 non-null   float64       
 4    put/call         1382 non-null   object        
 5    bid              1382 non-null   float64       
 6    ask              1382 non-null   float64       
 7    price            1382 non-null   float64       
 8    volume           1382 non-null   int64         
 9    open interest    1382 non-null   int64         
 10   implied vol      1283 non-null   float64       
 11   delta            1382 non-null   float64       
 12   gamma            1382 non-null   float64       
 13   rho              1382 non-null   float64       
 14   theta

In [93]:
options[0:10]

,symbol,under,expiration,strike,put/call,bid,ask,price,volume,open interest,implied vol,delta,gamma,rho,theta,vega,nonstd,expiration_dates
date,,,,,,,,,,,,,,,,,,
2008-03-14,GECC,GE,2008-03-22,31.0,C,2.90,2.96,2.93,297,6428,0.497979,0.902681,0.073764,0.005293,-0.028662,0.008058,0,2010-01-16
2008-03-14,GECF,GE,2008-03-22,30.0,C,3.85,4.00,4.00,15,8159,0.625794,0.923114,0.049219,0.005235,-0.030201,0.006756,0,2010-01-16
2008-03-14,GECG,GE,2008-03-22,35.0,C,0.19,0.20,0.20,6349,61878,0.322267,0.227791,0.200090,0.001440,-0.032560,0.014145,0,2010-01-16
2008-03-14,GECN,GE,2008-03-22,34.0,C,0.52,0.56,0.54,8099,24180,0.334112,0.463514,0.253875,0.002903,-0.044405,0.018607,0,2010-01-16
2008-03-14,GECO,GE,2008-03-22,36.0,C,0.06,0.07,0.07,913,11057,0.332842,0.091404,0.105374,0.000580,-0.018291,0.007694,0,2010-01-16
2008-03-14,GECR,GE,2008-03-22,39.0,C,0.00,0.02,0.02,0,3043,0.459030,0.013551,0.016140,0.000086,-0.005328,0.001625,0,2010-01-16
2008-03-14,GECS,GE,2008-03-22,37.5,C,0.01,0.02,0.02,47,43125,0.370896,0.023558,0.032008,0.000150,-0.006899,0.002604,0,2010-01-16
2008-03-14,GECZ,GE,2008-03-22,32.5,C,1.55,1.60,1.70,5745,20577,0.400615,0.771970,0.161054,0.004705,-0.040500,0.014153,0,2010-01-16
2008-03-14,GEDA,GE,2008-04-19,28.0,C,5.95,6.10,5.75,167,527,0.470208,0.914628,0.031704,0.023884,-0.010986,0.016350,0,2010-01-16


In [95]:
# filter options contracts for approx. 45 DTE
delta1 = timedelta(days=30)
delta2 = timedelta(days=60)

contracts_dte = pd.DataFrame()

i = 0

while i < len(options):
    dte1 = options.index[i] + delta1
    dte2 = options.index[i] + delta2
    contracts_dte = contracts_dte.append(options[(options['expiration_dates']>=dte1) & \
                          (options['expiration_dates']<=dte2)])
    i = i + 1

In [100]:
contracts_dte.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 0 entries
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0    symbol           0 non-null      object        
 1    under            0 non-null      object        
 2    expiration       0 non-null      object        
 3    strike           0 non-null      float64       
 4    put/call         0 non-null      object        
 5    bid              0 non-null      float64       
 6    ask              0 non-null      float64       
 7    price            0 non-null      float64       
 8    volume           0 non-null      int64         
 9    open interest    0 non-null      int64         
 10   implied vol      0 non-null      float64       
 11   delta            0 non-null      float64       
 12   gamma            0 non-null      float64       
 13   rho              0 non-null      float64       
 14   theta            0 non-null      f

In [ ]:
# todo: sell options contract when exit criteria met:
'''
Downtrend (Underlying's price < 50 day SMA)
 Exit the position, sell the Put, before expiration when:
    a. Put price drops 50%
    b. Underlying's price > previous day's High & %K < 40 (i.e. is oversold)
    c. Sell half of position if Put price gains 80%
    d. Sell at time T (maximum holding period with no gain or loss)

Uptrend (Underlying's price > 50 day SMA)
 Exit the position, sell the Call, before expiration when:
    a. Call price drops 50%
    b. Underlying's price < previous day's Low & %K > 60 (overbought)
    c. Sell half of position if Call price gains 80%
    d. Sell at time T (maximum holding period with no gain or loss)

    *Risk Management: Option Price * 100 <= 5% of trading capital
'''